In [ ]:
# Step 1: Data Acquisition
# Download the dataset file(GTSRB - German Traffic Sign Recognition Benchmark from Kaggle)

# Install kaggle
!pip install kaggle

# Mount the Google drive so that you can store your Kaggle API credentials
from google.colab import drive
drive.mount('/content/drive')

# Create a Kaggle directory
!mkdir -p ~/.kaggle

# Copy the kaggle.json file to the Kaggle directory
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/

# Set permissions for the Kaggle API key
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset using the Kaggle API
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

# Unzip the dataset (if needed)
!unzip gtsrb-german-traffic-sign.zip

In [15]:
# Step 2: Data Preprocessing
import os
import cv2
import numpy as np
import pandas as pd

def load_data(data_dir):
    images = []
    labels = []

    # Load the corresponding CSV file based on data_dir
    csv_file = os.path.join(f'{data_dir}.csv')  # Construct CSV filename
    data_df = pd.read_csv(csv_file)

    # Get the image file paths from the CSV
    image_paths = data_df['Path'].tolist()

    # Iterate over image paths and load images with corresponding labels
    for i, image_path in enumerate(image_paths):
        full_image_path = os.path.join(image_path) # Construct full image path
        image = cv2.imread(full_image_path)
        if image is not None:
            image = cv2.resize(image, (32, 32))
            images.append(image / 255.0)
            labels.append(data_df.loc[i, 'ClassId'])  # Extract label using index i from 'ClassId' column

    return np.array(images), np.array(labels)

train_images, train_labels = load_data('Train')
test_images, test_labels = load_data('Test')
print(f"Number of training images: {len(train_images)}")
print(f"Number of training labels: {len(train_labels)}")
print(f"Number of test images: {len(test_images)}")
print(f"Number of test labels: {len(test_labels)}")

Number of training images: 39209
Number of training labels: 39209
Number of test images: 12630
Number of test labels: 12630


In [23]:
# Adding sign descriptions corresponding to sign ids of 'meta.csv' file

sign_descriptions = {
    "1.32": "Pedestrian crossing",
    "3.29": "Speed limit (20km/h)",
    "3.27": "Speed limit (100km/h)",
    "1.22": "Right-of-way at the next intersection",
    "2.3": "Priority road",
    "2.1": "Yield",
    "2.2": "Stop",
    "3.1": "No entry",
    "3.3": "No vehicles",
    "3.21": "End of no passing",
    "1.39": "Traffic signals",
    "1.2": "Dangerous curve to the right",
    "1.1": "Dangerous curve to the left",
    "1.3.2": "Double curve (first to the right)",
    "1.13": "Slippery road",
    "1.5.2": "Road narrows on the right",
    "1.37": "Crosswind",
    "1.24": "Road work",
    "1.33": "Right-hand curve",
    "1.34": "Left-hand curve",
    "1.36": "Traffic merges from the right",
    "3.42": "End of all speed and passing limits",
    "4.2": "Mandatory direction (straight or right)",
    "4.3": "Mandatory direction (straight or left)",
    "4.1": "Mandatory direction (straight)",
    "4.4": "Mandatory direction (right)",
    "4.5": "Mandatory direction (left)",
    "4.7": "Roundabout",
    "4.8": "Mandatory path for bicycles",
    "3.26": "End of speed limit (80km/h)",
    "3.28": "End of prohibition of overtaking",
    "3.25": "End of speed limit (60km/h)",
    "3.3": "No vehicles",
}

# Load the Meta.csv file
meta_df = pd.read_csv('Meta.csv')

# Add the new column with sign descriptions(By converting 'SignId' column to 'str')
meta_df['SignDescription'] = meta_df['SignId'].astype(str).map(sign_descriptions)

# Save the updated DataFrame
meta_df.to_csv('meta_updated.csv', index=False)

In [ ]:
# Step 3 : Design a CNN architecture
# Define a CNN model using Tensorflow/Keras

import tensorflow as tf
from tensorflow.keras import layers, models # layers provides access to various neural network layers, models allow for model creation

# Create a sequential model where each layer has one input tensor and one output tensor
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(43, activation='softmax')  # 43 classes for traffic signs
])

# 1. Convolutional Layer: This adds a 2D convolutional layer with 32 filters (or kernels), each of size 3 × 3 3×3. The activation function used is ReLU (Rectified Linear Unit), which introduces non-linearity by outputting the input directly if it is positive; otherwise, it outputs zero. The input_shape parameter specifies the shape of the input images, which are grayscale images of size 28 × 28 28×28 pixels.
# 2. Max Pooling Layer: This layer downsamples the feature maps by taking the maximum value in each 2 × 2 2×2 window. This reduces the spatial dimensions of the feature maps by half, helping to decrease computational load and control overfitting by providing an abstracted representation of the features.
# 3. Second Convolutional Layer: Similar to the first convolutional layer but with 64 filters. This layer learns more complex features from the pooled outputs of the previous layer.
# 4. Second Max Pooling Layer: Again, this layer further reduces the size of the feature maps by taking the maximum value in 2 × 2 2×2 windows from the output of the last convolutional layer.
# 5. Flatten Layer: This layer converts the multi-dimensional output from the previous layer into a one-dimensional vector. It prepares the data for the fully connected (Dense) layers that follow. For example, if the output from the last pooling layer is 3 × 3 × 64 3×3×64, it will be flattened to a vector of length 576 576.
# 6. Dense Layer: A fully connected layer with 128 neurons. Each neuron receives input from all neurons in the previous layer. The ReLU activation function is again used here to introduce non-linearity.
# 7. Output Layer: This is another Dense layer but with 43 neurons corresponding to the number of classes in the traffic sign dataset. The softmax activation function is used here to convert raw logits into probabilities that sum to one across all classes. This allows for multi-class classification where each output neuron represents a class.

In [10]:
# Step 4: Compile the Model
# Compile the model with an appropriate optimizer and loss function
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 'Adam' optimizer, refers to adaptive moment estimation algorithm used,
# The loss function used during training is 'sparse_categrical_crossentropy'
# 'Accuracy' is the metric specified to be evaluated during training and testing

In [ ]:
# Step 5: Train and validate the model

# 5.1: Train the model
# Split the dataset into training and validation sets(eg: using 'train_test_split' from sklearn) and train the model
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size = 0.2)

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# It splits the original dataset into a training set and a validation set using an 80-20 split.
# It trains the Keras model on the training set for 10 epochs while validating its performance on the validation set after each epoch.

In [ ]:
# Step 6: Evaluating the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_accuracy:.4f}')

In [ ]:
# Step 7: Testing the model using an uploaded image
from google.colab import files
from IPython.display import Image, display, clear_output
import ipywidgets as widgets
from PIL import Image as PILImage

# Creating an upload button
uploader = widgets.FileUpload(
    accept='image/*',  # Accept only image files
    multiple=False      # Do not allow multiple files
)

display(uploader)

# Function that preprocess the uploaded image and uses the model to make predictions
def load_and_preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (32, 32))  # Resize to match model input shape
    img = img / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def predict_uploaded_image(model, uploader):
    if uploader.value:
        # Retrieve the uploaded file
        [uploaded_file] = uploader.value.values()

        # Get the content and the name of the uploaded file
        content = uploaded_file['content']
        name = uploaded_file['metadata']['name']

        # Write the content to a file
        with open(name, 'wb') as f:
            f.write(content)

        # Preprocess the image and make predictions
        img_array = load_and_preprocess_image(name)
        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction)

        # Get the sign description from the CSV file
        sign_description = meta_df[meta_df['ClassId'] == predicted_class]['SignDescription'].values
        sign_description = sign_description[0] if len(sign_description) > 0 else "Description not found"

        # Display the uploaded image and the prediction result
        pil_img = PILImage.open(name)
        pil_img.thumbnail((150, 150))  # Resize the image to 150x150 pixels
        display(pil_img)

        print(f'The model predicts that the image is of class value: {predicted_class}. {sign_description}')

        # Clear the uploader for the next upload
        uploader.value.clear()

# Creating a button for prediction

button = widgets.Button(description="Predict")
output = widgets.Output()

# Define the button click event
def on_button_clicked(b):
    with output:
        clear_output(wait=True)  # Clear previous output
        predict_uploaded_image(model, uploader)

button.on_click(on_button_clicked)

# Display button and output area
display(button, output)